In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("TN_IC_ECM_CRD_STAT_H_DONG.txt", sep = "\|", header = 0)
#df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
data = df[['`bs_yr_mon`', '`hang_cd`', '`stat_h_dong_027`', '`stat_h_dong_028`' ]]

In [3]:
data.columns = ['데이터 기준월', '행정동코드', '60대', '70대 이상']

In [4]:
print(data.dtypes)

데이터 기준월    object
행정동코드      object
60대        object
70대 이상     object
dtype: object


In [5]:
data = data.replace(to_replace ='\`', value = '', regex = True)  #결측치 -에서 0으로 바꾸기(문자면 오류)

In [6]:
#data type 변경
data.astype({'데이터 기준월' : 'int'})
data.astype({'행정동코드' : 'int'})
data.astype({'60대' : 'float'})
data.astype( {'70대 이상' : 'float'})
print(data.dtypes)

데이터 기준월    object
행정동코드      object
60대        object
70대 이상     object
dtype: object


In [7]:
#data type to_numeric
data['데이터 기준월'] = pd.to_numeric(data['데이터 기준월'])
data['행정동코드'] = pd.to_numeric(data['행정동코드'])
data['60대'] = pd.to_numeric(data['60대'])
data['70대 이상'] = pd.to_numeric(data['70대 이상'])
print(data.dtypes)

데이터 기준월      int64
행정동코드        int64
60대        float64
70대 이상     float64
dtype: object


In [8]:
#두 컬럼 합산
data['60대이상소득'] = (data['60대'] + data['70대 이상'])

In [9]:
#2018년 3개월 제외 -> 2019년 것으로만 정제
data = data[data['데이터 기준월'] >= 201901]

In [10]:
#미관측된 둔촌 1동의 행정동코드 추가
new_row = {'행정동코드' : 1125070, '데이터 기준월' : 201906.5, '60대' : 0, '70대 이상' : 0, '60대이상소득' :0  }
data = data.append( new_row, ignore_index = True )

In [11]:
data_m = data.groupby('행정동코드').mean()
#new_row = {'행정동코드' : 1125070, '데이터 기준월' : 201906.5, '60대' : 0, '70대 이상' : 0, '60대이상소득' :0  }
#data_m = data_m.append( new_row, ignore_index = True )
#['저소득노인합계'].loc[(df['동'] == '둔촌1동')] = 33 #결측된 둔촌 1동을 둔촌2동 데이터로 보정

In [12]:
df = pd.read_excel("7자리_행정동.xlsx", thousands = ',') #엑셀로 저장한거 1000단위의 , 제거하고 불러오기

In [13]:
del df['Unnamed: 0'] # Unnamed 0 생긴거 index에서 저장할때 생긴거 같은데 불필요하므로 제거

In [14]:
df.columns = ['구', '동', '행정동코드']

In [15]:
data_a= pd.merge(data_m, df, on="행정동코드", how='right')

In [16]:
#data_a

In [17]:
data_a['60대이상소득'].loc[(data_a['행정동코드'] == 1125070.0)] = 642.725000 #결측된 둔촌 1동을 둔촌2동 데이터로 보정
#data_a['60대이상소득'].loc[(data_a['행정동코드'] == 1125070.0)] = 642.5000 #결측된 둔촌 1동을 둔촌2동 데이터로 보정

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
#data_a

In [19]:
#column 추출
data_f = data_a[['구', '동', '60대이상소득']]

In [20]:
#rename
data_f.columns = [['구', '동', '행정동별평균소득']]

In [21]:
#data_f

In [22]:
#행정동코드 merge를 위한 xlsx 파일 로드
data = pd.read_excel("geojson_행정동코드.xlsx")

In [23]:
#data

In [24]:
del data['시']
#data

In [25]:
#data join
join_data_f= data_f.join(data, how = 'left')

In [26]:
#column rename
join_data_f.columns = [['구2', '동2', '행정동별평균소득', '구', '동', '행정동코드']]

In [27]:
#불필요한 column 제거
del join_data_f["구2"]
del join_data_f["동2"]
#join_data_f

In [28]:
join_data_f.columns

MultiIndex(levels=[['구', '구2', '동', '동2', '행정동별평균소득', '행정동코드']],
           codes=[[4, 0, 2, 5]])

In [29]:
#column rename
join_data_f = join_data_f[['행정동코드', '구', '동', '행정동별평균소득']]

In [30]:
#join_data_f.sort_values(by = [ '행정동별평균소득' , '행정동코드'], ascending = False).head(10)

In [31]:
#to_excel
join_data_f.to_excel('소득.xlsx', encoding = 'utf-8')

In [32]:
import sklearn
sklearn.__version__

import pandas as pd
import numpy as np
import folium

from sklearn.preprocessing import MinMaxScaler

In [33]:
import pandas as pd
import numpy as np
df = pd.read_excel("소득.xlsx", thousands = ',', ) #엑셀로 저장한거 1000단위의 , 제거하고 불러오기
#df


In [34]:
del df['Unnamed: 0'] # Unnamed 0 생긴거 index에서 저장할때 생긴거 같은데 불필요하므로 제거

In [35]:
df = df.drop(0,0)

In [36]:
#df

In [37]:
scaler= MinMaxScaler()

df[["행정동별평균소득"]]= scaler.fit_transform(df[["행정동별평균소득"]])

#df

In [38]:
low = df.sort_values(by = ['행정동별평균소득', '행정동코드'], ascending = True).head(10)
#abc

In [39]:
high = df.sort_values(by = ['행정동별평균소득', '행정동코드'], ascending = False).head(10)
#haha

In [40]:
#동별 평균소득 하위 10개
low = low[['행정동코드', '구', '동']]
low

,행정동코드,구,동
199,11410690.0,서대문구,남가좌1동
6,11110580.0,종로구,교남동
14,11110700.0,종로구,숭인1동
232,11470610.0,양천구,신월6동
11,11110670.0,종로구,창신1동
24,11140605.0,중구,을지로동
13,11110690.0,종로구,창신3동
406,11710647.0,송파구,위례동
283,11545690.0,금천구,시흥3동
165,11350621.0,노원구,중계1동


In [41]:
#동별 평균소득 상위 10개
high = high[['행정동코드', '구', '동']]
high

,행정동코드,구,동
365,11680640.0,강남구,역삼1동
330,11620695.0,관악구,신림동
336,11620595.0,관악구,청룡동
66,11215710.0,광진구,화양동
206,11410615.0,서대문구,연희동
359,11680521.0,강남구,논현1동
211,11440660.0,마포구,서교동
259,11500540.0,강서구,화곡1동
321,11620575.0,관악구,행운동
305,11590530.0,동작구,상도1동


In [42]:
df.to_excel('소득_정규화.xlsx', encoding = 'utf-8')